In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
vc1 = pd.read_csv('../input/solar-gen/m2015.csv', error_bad_lines=False)
vc2 = pd.read_csv('../input/solar-gen/m2016.csv', error_bad_lines=False)
vc3 = pd.read_csv('../input/solar-gen/m2016.csv', error_bad_lines=False)
#vc3['RefCell1_Wm2_Avg']=vc3['RefCell1_Wm2_Avg']/3
#c1['RefCell1_Wm2_Avg']=vc1['RefCell1_Wm2_Avg']/2


In [ ]:
vc1 = pd.read_csv('../input/solar-gen/m2015.csv', error_bad_lines=False)
vc2 = pd.read_csv('../input/solar-gen/m2016.csv', error_bad_lines=False)
vc3 = pd.read_csv('../input/solar-gen/m2016.csv', error_bad_lines=False)
#vc3['RefCell1_Wm2_Avg']=vc3['RefCell1_Wm2_Avg']/3
#c1['RefCell1_Wm2_Avg']=vc1['RefCell1_Wm2_Avg']/2
vtnt=pd.concat([vc1,vc2,vc3],axis=0)

vtnt.isnull().any()

vtnt.pop('WindRef_V_Min')
vtnt.pop('WindSpeedAve_ms')

btnt=vtnt.dropna()
X = btnt.iloc[:, 0:-1]
y = btnt.iloc[:, -1]
X = X.values
y = y.values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,shuffle=False)


In [ ]:
#btnt.isNaN().any()

In [ ]:
# lstm autoencoder recreate sequence
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Dropout
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

# define input sequence
X=btnt.iloc[:,:]
bnt=X.values
# reshape input into [samples, timesteps, features]
n_in = len(bnt)

# define model


In [ ]:
X = btnt.iloc[:, 0:-1]
y = btnt.iloc[:, -1]
X = X.values
y = y.values


In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,shuffle=False)

from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# load dataset


NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))
NN_model.add(Dropout(0.2))
# The Hidden Layers :
NN_model.add(Dense(256, activation='relu'))
NN_model.add(Dropout(0.2))
NN_model.add(Dense(256,activation='relu'))
NN_model.add(Dropout(0.2))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dropout(0.2))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='relu'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

In [ ]:
import pickle
pkl_filename = "baisc.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

NN_model.fit(X_train, y_train, epochs=100, batch_size=1440, validation_split = 0.2, callbacks=callbacks_list)


In [ ]:
btnt

In [ ]:
wights_file = './Weights-068--205.54189.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
y_pred=NN_model.predict(X_test)
a=np.sum(y_test, dtype = np.float32)
np.sum(y_pred, dtype = np.float32)
for i in range(0,90):
    X_test[:,-2]=i
    #X_test=np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    y_pred = NN_model.predict(X_test)
    sum=np.sum(y_pred, dtype = np.float32)
    
    if sum>a:
        a=sum
        angle=i
        print(a)
        print(i)

In [ ]:
X_test[:,-2]=16
#X_test=np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
y_pred = NN_model.predict(X_test)
sum=np.sum(y_pred, dtype = np.float32)
    

In [ ]:
sum

In [ ]:
X_train.shape[1]

In [ ]:

model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mean_squared_error', optimizer='adam')
#model.fit(X_train, y_train, epochs=100, batch_size=1440, validation_split = 0.2)
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=1440, verbose=0)

In [ ]:
model.summary()

In [ ]:
estimator

In [ ]:
# Regression Example With Boston Dataset: Baseline
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=1440, verbose=0)



In [ ]:
estimator.fit(X_train, y_train, epochs=100, batch_size=1440, validation_split = 0.1)